In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

C:\Users\EXCELL  COMPUTERS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\EXCELL  COMPUTERS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv(r"C:\Users\EXCELL  COMPUTERS\Programming_Data_Science\CSV_Files\Titanic.csv")

In [3]:
data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
623,624,0,3,"Hansen, Mr. Henry Damsgaard",male,21.0,0,0,350029,7.8542,NaN,S
828,829,1,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q
754,755,1,2,"Herman, Mrs. Samuel (Jane Laver)",female,48.0,1,2,220845,65.0000,NaN,S
284,285,0,1,"Smith, Mr. Richard William",male,NaN,0,0,113056,26.0000,A19,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [4]:
#Drop the irlevent column to practice pipeline
data.drop(columns=['Name','Ticket','Cabin'],axis = 1,inplace = True)

In [5]:
data.sample(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
816,817,0,3,female,23.0,0,0,7.925,S
350,351,0,3,male,23.0,0,0,9.225,S
790,791,0,3,male,NaN,0,0,7.750,Q


In [6]:
x_train,x_test,y_train,y_test = train_test_split(data.drop(columns=['Survived']),data['Survived'],test_size = 0.2,random_state = 42)

In [7]:

label_encoder = LabelEncoder()
x_train['Sex'] = label_encoder.fit_transform(x_train['Sex'])
x_train['Embarked'] = label_encoder.fit_transform(x_train['Embarked'])


In [8]:
x_train.sample(3)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
679,680,1,1,36.0,0,1,512.3292,0
64,65,1,1,NaN,0,0,27.7208,0
633,634,1,1,NaN,0,0,0.0000,2


In [9]:
x_train.drop(columns=['PassengerId'],axis = 1,inplace = True)
x_test.drop(columns=['PassengerId'],axis = 1,inplace = True)

In [10]:
#Imputation transfromers
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder = 'passthrough')

In [11]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 331 to 102
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    int32  
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    int32  
dtypes: float64(2), int32(2), int64(3)
memory usage: 38.9 KB


In [12]:

trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')

In [13]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
],remainder = 'passthrough')

In [14]:
trf4 = SelectKBest(score_func=chi2,k=8)

In [15]:
trf5 = DecisionTreeClassifier()

In [16]:
#Creating the pipeline

In [17]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [18]:
pipe.fit(x_train,y_train)

C:\Users\EXCELL  COMPUTERS\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001C71D806F70>)),
                ('trf5', DecisionTreeClassifier())])

In [35]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [37]:

label_encoder = LabelEncoder()
x_test['Sex'] = label_encoder.fit_transform(x_test['Sex'])
x_test['Embarked'] = label_encoder.fit_transform(x_test['Embarked'])


In [40]:
y_pred = pipe.predict(x_test)

In [39]:
from sklearn.metrics import accuracy_score

In [41]:
accuracy_score(y_pred,y_test)

0.7877094972067039